In [64]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from keras.utils import pad_sequences

In [65]:
df = pd.read_csv('./output/data/features.csv')

In [66]:
df.columns

Index(['file_path', 'sampling_rate', 'label', 'mfcc_1_mean', 'mfcc_2_mean',
       'mfcc_3_mean', 'mfcc_4_mean', 'mfcc_5_mean', 'mfcc_6_mean',
       'mfcc_7_mean', 'mfcc_8_mean', 'mfcc_9_mean', 'mfcc_10_mean',
       'mfcc_11_mean', 'mfcc_12_mean', 'mfcc_13_mean', 'chroma_1_mean',
       'chroma_2_mean', 'chroma_3_mean', 'zcr_mean',
       'spectral_contrast_1_mean', 'spectral_contrast_2_mean',
       'spectral_contrast_3_mean', 'spectral_contrast_4_mean',
       'spectral_contrast_5_mean', 'spectral_contrast_6_mean',
       'spectral_centroid_mean', 'spectral_bandwidth_mean',
       'spectral_rolloff_mean', 'lpc_1_mean', 'lpc_2_mean', 'lpc_3_mean',
       'lpc_4_mean', 'lpc_5_mean', 'lpc_6_mean', 'lpc_7_mean', 'lpc_8_mean',
       'lpc_9_mean', 'lpc_10_mean', 'lpc_11_mean', 'lpc_12_mean'],
      dtype='object')

In [67]:
# Drop the 'file_path' column
data = df.drop(columns=['file_path'])

# Convert 'label' column values from 'fake' and 'real' to 0 and 1
data['label'] = data['label'].replace({'fake': 0, 'real': 1})

# Display the updated dataframe
data.head()

/var/folders/gy/d7p0zrqx7l3f7dwplj0g_b040000gn/T/ipykernel_16846/3452172015.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['label'] = data['label'].replace({'fake': 0, 'real': 1})


,sampling_rate,label,mfcc_1_mean,mfcc_2_mean,mfcc_3_mean,mfcc_4_mean,mfcc_5_mean,mfcc_6_mean,mfcc_7_mean,mfcc_8_mean,...,lpc_3_mean,lpc_4_mean,lpc_5_mean,lpc_6_mean,lpc_7_mean,lpc_8_mean,lpc_9_mean,lpc_10_mean,lpc_11_mean,lpc_12_mean
0,24000,0,-392.408060,122.706493,4.529471,-1.131724,10.133050,-7.359573,-12.236763,-11.748826,...,0.206947,0.242434,0.083058,-0.000745,0.017267,0.048853,0.049655,0.011606,-0.013775,0.056669
1,24000,1,-288.633987,150.553761,-51.823300,9.575870,16.140331,-25.634484,-9.063233,-13.583262,...,0.942317,0.165730,-0.168018,-0.083517,0.020074,0.079114,0.045597,-0.013818,-0.022306,0.027391
2,24000,1,-321.349296,128.728553,-45.678164,16.015681,2.248679,-36.931314,-16.453398,-21.782121,...,0.791051,0.180148,-0.195515,-0.059862,0.091892,0.056669,-0.024036,-0.031334,0.023492,0.031977
3,24000,1,-300.014392,139.610539,-58.775005,-17.863993,-27.983239,-31.165856,-15.891464,-29.907360,...,0.625773,0.182251,-0.158980,-0.015436,0.072798,0.026417,0.033397,0.026217,-0.038438,0.072341
4,24000,0,-313.237382,128.302576,-47.012209,19.925841,13.332181,-29.089705,-13.109960,-17.376589,...,0.469962,0.320646,-0.054736,-0.086741,0.068926,0.095494,0.023775,-0.018482,0.001961,0.045553


In [68]:
data.shape

(3000, 40)

In [69]:
# 1. Load and preprocess data
features = data[['sampling_rate', 'mfcc_1_mean', 'mfcc_2_mean',
       'mfcc_3_mean', 'mfcc_4_mean', 'mfcc_5_mean', 'mfcc_6_mean',
       'mfcc_7_mean', 'mfcc_8_mean', 'mfcc_9_mean', 'mfcc_10_mean',
       'mfcc_11_mean', 'mfcc_12_mean', 'mfcc_13_mean', 'chroma_1_mean',
       'chroma_2_mean', 'chroma_3_mean', 'zcr_mean',
       'spectral_contrast_1_mean', 'spectral_contrast_2_mean',
       'spectral_contrast_3_mean', 'spectral_contrast_4_mean',
       'spectral_contrast_5_mean', 'spectral_contrast_6_mean',
       'spectral_centroid_mean', 'spectral_bandwidth_mean',
       'spectral_rolloff_mean', 'lpc_1_mean', 'lpc_2_mean', 'lpc_3_mean',
       'lpc_4_mean', 'lpc_5_mean', 'lpc_6_mean', 'lpc_7_mean', 'lpc_8_mean',
       'lpc_9_mean', 'lpc_10_mean', 'lpc_11_mean', 'lpc_12_mean']] # Select features
labels = data['label']

In [70]:
def create_sequences(features, sequence_length, overlap):
    """
    Creates sequences from feature data.

    Args:
        features: A NumPy array of shape (num_samples, num_features).
        sequence_length: The desired length of each sequence.
        overlap: The number of overlapping features between consecutive sequences.

    Returns:
        A tuple containing:
            - A NumPy array of shape (num_sequences, sequence_length, num_features).
            - A list of indices indicating the starting sample of each sequence.
    """
    sequences = []
    indices = []  # Store the starting index of each sequence
    for i in range(0, len(features) - sequence_length + 1, sequence_length - overlap):
        sequences.append(features[i: i + sequence_length])
        indices.append(i)  # Store the starting index
    return np.array(sequences), indices

In [71]:
# 1. Create sequences
sequence_length = 10  # Example value, adjust as needed
overlap = 5  # Example value, adjust as needed
sequences, indices = create_sequences(features, sequence_length, overlap)
labels = labels[indices]

# 2. Pad sequences
padded_sequences = pad_sequences(
    sequences, maxlen=sequence_length, padding="pre", truncating="post"
)

# Normalize features
scaler = MinMaxScaler()
# Reshape padded_sequences for scaling
num_samples, seq_len, num_features = padded_sequences.shape
padded_sequences_reshaped = padded_sequences.reshape(
    num_samples, seq_len * num_features
)

# Fit and transform the scaler on the reshaped data
padded_sequences_scaled = scaler.fit_transform(padded_sequences_reshaped)

# Reshape the scaled data back to the original shape
padded_sequences = padded_sequences_scaled.reshape(
    num_samples, seq_len, num_features
)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, labels, test_size=0.2, random_state=42
)

# 2. Create LSTM model
model = Sequential()
model.add(LSTM(units=64, input_shape=(sequence_length, 40)))
model.add(Dense(1, activation="sigmoid"))

# 3. Compile model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# 4. Train model with early stopping
early_stopping = EarlyStopping(monitor="val_loss", patience=5)
model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stopping],
)

# 5. Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/50


/Users/ronan/Developer/deepfake-audio-detector/venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.4837 - loss: 0.6951 - val_accuracy: 0.6042 - val_loss: 0.6771
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4921 - loss: 0.6952 - val_accuracy: 0.6042 - val_loss: 0.6753
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5545 - loss: 0.6868 - val_accuracy: 0.6042 - val_loss: 0.6805
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5292 - loss: 0.6916 - val_accuracy: 0.4167 - val_loss: 0.6932
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5037 - loss: 0.6918 - val_accuracy: 0.6875 - val_loss: 0.6822
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.5102 - loss: 0.6903 - val_accuracy: 0.6042 - val_loss: 0.6714
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5244 - loss: 0.6880 - val_accuracy: 0.6667 - val_loss: 0.6788
Epoch 8/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5613 - loss: 0.6902 - val_accuracy: 0.7500 - val_loss: 0.